In [1]:
import os
import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
data_dir = "/root/20242R0136COSE47402/FinalProject/data/test"
class_candidate = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
text_inputs = []
for folder in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, folder)):
        folder = folder.replace('_', ' ')
        text_inputs.append(f"a photo of {folder}")

In [14]:
image_paths = []
image_labels = []

for class_name in class_candidate:
    class_folder = os.path.join(data_dir, class_name)
    for img_name in os.listdir(class_folder):
        if img_name[0] == '.':
            continue
        img_path = os.path.join(class_folder, img_name)
        image_paths.append(img_path)
        class_name = class_name.replace('_', ' ')
        image_labels.append(f"a photo of {class_name}")
        

print(f"test dataset size : {len(image_paths)}")

test dataset size : 25250


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [16]:
ans = 0
model.eval()

for idx, img_path in enumerate(image_paths):
    image = Image.open(img_path).convert('RGB')
    inputs = processor(text=text_inputs, images=image, return_tensors='pt', padding=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)

    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    if text_inputs[torch.argmax(probs).item()] == image_labels[idx]:
        ans += 1
    if idx % 100 == 0:
        print(f"# {idx} finished.")

precision = ans / len(image_paths)
precision


# 0 finished.
# 100 finished.
# 200 finished.
# 300 finished.
# 400 finished.
# 500 finished.
# 600 finished.
# 700 finished.
# 800 finished.
# 900 finished.
# 1000 finished.
# 1100 finished.
# 1200 finished.
# 1300 finished.
# 1400 finished.
# 1500 finished.
# 1600 finished.
# 1700 finished.
# 1800 finished.
# 1900 finished.
# 2000 finished.
# 2100 finished.
# 2200 finished.
# 2300 finished.
# 2400 finished.
# 2500 finished.
# 2600 finished.
# 2700 finished.
# 2800 finished.
# 2900 finished.
# 3000 finished.
# 3100 finished.
# 3200 finished.
# 3300 finished.
# 3400 finished.
# 3500 finished.
# 3600 finished.
# 3700 finished.
# 3800 finished.
# 3900 finished.
# 4000 finished.
# 4100 finished.
# 4200 finished.
# 4300 finished.
# 4400 finished.
# 4500 finished.
# 4600 finished.
# 4700 finished.
# 4800 finished.
# 4900 finished.
# 5000 finished.
# 5100 finished.
# 5200 finished.
# 5300 finished.
# 5400 finished.
# 5500 finished.
# 5600 finished.
# 5700 finished.
# 5800 finished.
# 5900 fi

0.8978613861386139